## This compares to DoubleBracketIteration whenever possible

In [4]:
from qibo.hamiltonians import SymbolicHamiltonian
from qibo import symbols
from double_bracket_evolution_oracles import *
from group_commutator_iteration_transpiler import *
from numpy.linalg import norm
def test_dbi_evolution_oracle(t_step, eps):

    
    h_x = SymbolicHamiltonian( symbols.X(0) + symbols.Z(0) * symbols.X(1) + symbols.Y(2) 
                              + symbols.Y(1) * symbols.Y(2), nqubits = 3 )
    d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits = 3 )
    h_input = h_x + d_0
    

    evolution_oracle = EvolutionOracle(h_input, "ZX",
                        mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)
    
    evolution_oracle.eps_trottersuzuki = eps
    
    U_hamiltonian_simulation = evolution_oracle.circuit(t_step).unitary()
    V_target = h_input.exp(t_step)
    
    assert norm(U_hamiltonian_simulation-V_target) < eps
    
test_dbi_evolution_oracle( 1, 1e-3)    

[Qibo 0.2.5|WARNING|2024-04-04 06:37:37]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


`DoubleBracketRotationType.group_commutator_other_sorting` is the same as currently `DoubleBracketIteration` group commutator

In [24]:
from double_bracket import *

def test_group_commutator_other_sorting_dbi_vs_gci(t_step, eps):

    
    h_x = SymbolicHamiltonian( symbols.X(0) + symbols.Z(0) * symbols.X(1) + symbols.Y(2) 
                              + symbols.Y(1) * symbols.Y(2), nqubits = 3 )
    d_0 = SymbolicHamiltonian(symbols.Z(0), nqubits = 3 )
    h_input = h_x + d_0    



    dbi = DoubleBracketIteration(deepcopy(h_input.dense))
    dbi.mode = DoubleBracketGeneratorType.group_commutator

    def wrapper_gc(self,step, d):
        return (
                    self.h.exp(-step)
                    @ self.backend.calculate_matrix_exp(-step, d)
                    @ self.h.exp(step)
                    @ self.backend.calculate_matrix_exp(step, d)
                )
    V_dbi = wrapper_gc(dbi, np.sqrt(t_step), d_0.dense.matrix)
    
    evolution_oracle = EvolutionOracle(h_input, "ZX",
                        mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)    
    evolution_oracle.eps_trottersuzuki = eps
    
    evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)
    
    gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle ))
    gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting
    
    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )   
    U_gci = unitary_gc_from_oracles['forwards']
    
    assert norm(U_gci.unitary() - unitary_gc_from_oracles['backwards'].unitary().conj().T) < 1e-12
    
    assert norm(U_gci.unitary() - V_dbi ) < 5*eps
    
    gci.input_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
    evolution_oracle_diagonal_target.mode_evolution_oracle = EvolutionOracleType.numerical
    
    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )   
    U_gci = unitary_gc_from_oracles['forwards']
    
    assert norm(U_gci - unitary_gc_from_oracles['backwards'].conj().T) < 1e-12
    
    assert norm(U_gci - V_dbi ) < 5*eps
    
    gci.input_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.text_strings
    evolution_oracle_diagonal_target.mode_evolution_oracle = EvolutionOracleType.text_strings
    
    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )   
    U_gci = unitary_gc_from_oracles['forwards']
    
    assert isinstance(U_gci, str)

    
test_group_commutator_dbi_vs_gci(.1, 1e-5)

[Qibo 0.2.5|WARNING|2024-04-04 06:55:29]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.
[Qibo 0.2.5|WARNING|2024-04-04 06:55:29]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


# Show that double bracket iteration group commutator and dbi converge for small s BHMM




# Show that double bracket iteration group commutator and gci are numerically exact



# Show that double bracket iteration  and gci gc and gc_reduced converge for small s BHMM




In [8]:
dbi(t_step, d = d_0.dense.matrix)


#### 2. Evolution oracle hamiltonian simulation


In [9]:
evolution_oracle_hamiltonian_simulation = EvolutionOracle(deepcopy(h_input), "ZX",
                               mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)


In [10]:
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)



In [11]:
t_step = 1
def check_hs_eps(eps):
    evolution_oracle_hamiltonian_simulation.eps_trottersuzuki = eps
    U_hamiltonian_simulation = evolution_oracle_hamiltonian_simulation.circuit(t_step).unitary()
    V_target = h_input.exp(t_step)
    print(eps,norm(U_hamiltonian_simulation-V_target))

In [12]:
check_hs_eps(0.1)

0.1 0.08786496042239604


In [13]:
check_hs_eps(.001)

0.001 0.0003373755363091052


In [16]:
check_hs_eps(1e-4)

0.0001 8.43397274693628e-05


In [17]:

gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle_hamiltonian_simulation ))
gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting


In [18]:
gci.iterated_hamiltonian_evolution_oracle.h

In [12]:
unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

In [13]:
for a in DoubleBracketRotationType:
    print(a.name)

single_commutator
group_commutator
group_commutator_other_sorting
group_commutator_reduced
group_commutator_imperfect
group_commutator_reduced_imperfect


In [14]:
def test_gc_numerical_vs_circuit(eps):
    gci.iterated_hamiltonian_evolution_oracle.eps_trottersuzuki = eps
    evolution_oracle_diagonal_target.eps_trottersuzuki = eps
    
    gci.iterated_hamiltonian_evolution_oracle.please_be_verbose = False
    evolution_oracle_diagonal_target.please_be_verbose = False
    

    gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_reduced

    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.numerical

    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

    unitary_gc_existing = wrapper_gc(dbi, np.sqrt(t_step),d_0.dense.matrix)

    bckwd = unitary_gc_from_oracles['backwards']
    fwd = unitary_gc_from_oracles['forwards']

    e1 = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step))
    e2 = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step))

    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.hamiltonian_simulation

    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

    unitary_gc_existing = wrapper_gc(dbi, np.sqrt(t_step),d_0.dense.matrix)

    bckwdhs = unitary_gc_from_oracles['backwards'].unitary()
    fwdhs = unitary_gc_from_oracles['forwards'].unitary()

    e1hs = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step)).unitary()
    e2hs = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step)).unitary()

    print("e1:", norm( e1hs-e1))
    print("e2:", norm(e2hs-e2))
    print("Testing inversion forwards:", norm(bckwd-bckwdhs))
    print("Testing inversion backwards:", norm(fwd - fwdhs))

test_gc_numerical_vs_circuit(0.1)

test_gc_numerical_vs_circuit(0.0001)

e1: 0.08786496042239604
e2: 6.280369834735101e-16
Testing inversion forwards: 0.08786496042239617
Testing inversion backwards: 0.08786496042239596
e1: 8.43397274693628e-05
e2: 6.280369834735101e-16
Testing inversion forwards: 8.433972746973065e-05
Testing inversion backwards: 8.433972746997873e-05


In [26]:
def test_group_commutator_against_itself(gci, evolution_diagonal,eps = 0.0001):
    gci.iterated_hamiltonian_evolution_oracle.eps_trottersuzuki = eps
    evolution_oracle_diagonal_target.eps_trottersuzuki = eps
    for s in np.linspace(0,1,5):
        gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
        evolution_diagonal.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
        gc_hs = gci.group_commutator(s,evolution_diagonal)
        gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
        evolution_diagonal.mode_evolution_oracle = EvolutionOracleType.numerical
        gc_np = gci.group_commutator(s,evolution_diagonal)
        print(norm(gc_np['forwards']-gc_hs['forwards'].unitary()))
        print(norm(gc_np['backwards']-gc_hs['backwards'].unitary()))

gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator        
test_group_commutator_against_itself(gci, evolution_oracle_diagonal_target)

gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting        
test_group_commutator_against_itself(gci, evolution_oracle_diagonal_target)

gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_reduced      
test_group_commutator_against_itself(gci, evolution_oracle_diagonal_target)

0.0
0.0
1.1365570924547864e-05
1.1365570924508396e-05
3.178573543740718e-05
3.178573543738423e-05
2.331929054986953e-05
2.3319290550395577e-05
5.051045765082015e-05
5.051045765141829e-05
0.0
0.0
1.1365570924450749e-05
1.1365570924571942e-05
3.178573543667143e-05
3.178573543684212e-05
2.3319290550074785e-05
2.3319290550369457e-05
5.051045765037518e-05
5.051045764960636e-05
0.0
0.0
3.67220292448543e-05
3.672202924482476e-05
6.462333424612395e-05
6.462333424677995e-05
4.47483584471007e-05
4.474835844733171e-05
8.433972746997873e-05
8.433972746973065e-05


In [ ]:
def test_gc_numerical_vs_bracket(eps = 0.3):
    gci.iterated_hamiltonian_evolution_oracle.eps_trottersuzuki = eps
    evolution_oracle_diagonal_target.eps_trottersuzuki = eps
    
    gci.iterated_hamiltonian_evolution_oracle.please_be_verbose = False
    evolution_oracle_diagonal_target.please_be_verbose = False
    

    gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator

    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.numerical

    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

    import scipy
    unitary_gc_existing1 = scipy.linalg.expm( t_step * (d_0.dense.matrix @ h_x.dense.matrix
                                         -h_x.dense.matrix@d_0.dense.matrix))

    bckwd = unitary_gc_from_oracles['backwards']
    fwd = unitary_gc_from_oracles['forwards']

    e1 = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step))
    e2 = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step))

    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.hamiltonian_simulation

    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

    unitary_gc_existing = wrapper_gc(dbi, np.sqrt(t_step),-d_0.dense.matrix)

    bckwdhs = unitary_gc_from_oracles['backwards'].unitary()
    fwdhs = unitary_gc_from_oracles['forwards'].unitary()

    e1hs = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step)).unitary()
    e2hs = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step)).unitary()
    print("Test:", norm(unitary_gc_existing-unitary_gc_existing1))
    print("e1:", norm( e1hs-e1))
    print("e2:", norm(e2hs-e2))
    print("Testing inversion forwards:", norm(bckwd-bckwdhs))
    print("Testing inversion backwards:", norm(fwd - fwdhs))

test_gc_numerical_vs_bracket(0.1)

test_gc_numerical_vs_bracket(0.0001)

In [34]:
def gc_hs_eps(eps):
    gci.iterated_hamiltonian_evolution_oracle.eps_trottersuzuki = eps
    evolution_oracle_diagonal_target.eps_trottersuzuki = eps
    
    gci.iterated_hamiltonian_evolution_oracle.please_be_verbose = True
    evolution_oracle_diagonal_target.please_be_verbose = False
    
    gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
    evolution_oracle_diagonal_target.mode_evolution_oracle  = EvolutionOracleType.numerical

    gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator
    
    unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
               diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )
    
    unitary_gc_existing = wrapper_gc(dbi, np.sqrt(t_step),-d_0.dense.matrix)
    
    bckwd = unitary_gc_from_oracles['backwards']
    fwd = unitary_gc_from_oracles['forwards']
    
    e1 = gci.iterated_hamiltonian_evolution_oracle.circuit(np.sqrt(t_step))
    e2 = evolution_oracle_diagonal_target.circuit(np.sqrt(t_step))
    
    print("Backwards:", norm( bckwd - unitary_gc_existing))
    print("Forwards:", norm(fwd - unitary_gc_existing))
    print("Testing inversion forwards:", norm(fwd - e2.conj().T @ e1.conj().T @ e2 @e1))
    print("Testing inversion backwards:", norm(bckwd - e2.conj().T @ e1.conj().T @ e2 @e1))
    print("Testing inversion forwards:", norm(fwd - e1.conj().T @ e2.conj().T @  e1 @e2))
    print("Testing inversion backwards:", norm(bckwd - e1.conj().T @ e2.conj().T @  e1 @e2))
    print("Testing inversion forwards:", norm(fwd - e1@e2@ e1.conj().T @ e2.conj().T ))
    print("Testing inversion backwards:", norm(bckwd - e1@e2@e1.conj().T @ e2.conj().T ))
    print("Testing inversion forwards:", norm(fwd - e1.conj().T@e2@ e1 @ e2.conj().T ))
    print("Testing inversion backwards:", norm(bckwd - e1.conj().T@e2@e1 @ e2.conj().T ))
    print("Testing reversal:", norm(bckwd@fwd - unitary_gc_existing @unitary_gc_existing.T.conj() ))
    print("Testing reversal:", norm(bckwd@fwd - e1@e1.conj().T ))

In [35]:
gc_hs_eps(0.1)

Backwards: 4.705144430384231
Forwards: 0.001440854728059234
Testing inversion forwards: 3.30970601135848
Testing inversion backwards: 3.3448906768858175
Testing inversion forwards: 3.344890676885818
Testing inversion backwards: 3.3097060113584806
Testing inversion forwards: 3.2769939636040615
Testing inversion backwards: 4.146762372604833
Testing inversion forwards: 1.8387684917953713e-15
Testing inversion backwards: 4.705576215718977
Testing reversal: 3.759252603260064e-15
Testing reversal: 3.632964118172524e-15


We may improve the discrepancy by setting smaller eps

In [ ]:
gc_hs_eps(0.0001)

In [ ]:
norm(unitary_gc_from_oracles['forwards'].unitary() - unitary_gc_existing)

In [ ]:
norm(unitary_gc_from_oracles['backwards'].unitary() - unitary_gc_existing)

In [ ]:
stop

#### Test more explicitly


In [ ]:
u_h = gci.iterated_hamiltonian_evolution_oracle.circuit( np.sqrt(t_step)).unitary()
u_d = evolution_oracle_diagonal_target.circuit( np.sqrt(t_step)).unitary()
u_h_reversed = gci.iterated_hamiltonian_evolution_oracle.circuit( -np.sqrt(t_step)).unitary()
u_d_reversed = evolution_oracle_diagonal_target.circuit( -np.sqrt(t_step)).unitary()
norm( u_h_reversed @ u_d_reversed @ u_h @ u_d - unitary_gc_existing)

In [ ]:
u_h = gci.iterated_hamiltonian_evolution_oracle.circuit( np.sqrt(t_step))
u_d = evolution_oracle_diagonal_target.circuit( np.sqrt(t_step))
u_h_reversed = gci.iterated_hamiltonian_evolution_oracle.circuit( -np.sqrt(t_step))
u_d_reversed = evolution_oracle_diagonal_target.circuit( -np.sqrt(t_step))
norm( (u_h_reversed + u_d_reversed + u_h + u_d).unitary() - unitary_gc_existing)

#### 3. Evolution oracle numpy


In [ ]:
evolution_oracle_numerical = EvolutionOracle(deepcopy(h_input), "ZX",
                               mode_evolution_oracle = EvolutionOracleType.numerical)

gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle_numerical ))

In [ ]:
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.numerical)


In [ ]:
unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )

Compared to the group commutator using Hamiltonian simulation there will be small deviations that arise from Trotter-Suzuki decomposition

In [ ]:
norm(unitary_gc_from_oracles['backwards'] - unitary_gc_existing)

In [ ]:
norm(unitary_gc_from_oracles['forwards'] - unitary_gc_existing)

We may check by switching the group commutator flag that the difference comes from ordering and inversions

In [ ]:
gci.mode_double_bracket_rotation = DoubleBracketRotationType.group_commutator_other_sorting

unitary_gc_from_oracles = gci.group_commutator( np.sqrt(t_step),
           diagonal_association_evolution_oracle = evolution_oracle_diagonal_target )
norm(unitary_gc_from_oracles['forwards'] - unitary_gc_existing)

#### 4. Check gci rotation

In [ ]:
evolution_oracle_hamiltonian_simulation = EvolutionOracle(deepcopy(h_input), "ZX",
                               mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)
evolution_oracle_diagonal_target =  EvolutionOracle(d_0, "D0",
               mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(evolution_oracle_hamiltonian_simulation ))

In [ ]:
type(gci.iterated_hamiltonian_evolution_oracle)

In [ ]:
gci(t_step, diagonal_association=evolution_oracle_diagonal_target)

In [ ]:
type(gci.iterated_hamiltonian_evolution_oracle)

In [ ]:
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle

In [ ]:
u_frame_shifted = gci.iterated_hamiltonian_evolution_oracle.circuit(t_step).unitary()

In [ ]:
norm( dbi.h.exp(t_step) - u_frame_shifted)

In [ ]:
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle


In [ ]:
gci(t_step, diagonal_association=evolution_oracle_diagonal_target)

In [ ]:
dbi(t_step, d = evolution_oracle_diagonal_target.h.dense.matrix)

In [ ]:
u_frame_shifted = gci.iterated_hamiltonian_evolution_oracle.circuit(t_step).unitary()


In [ ]:
norm( dbi.h.exp(t_step) - u_frame_shifted)

In [ ]:
for k in range(3):
    gci(t_step, diagonal_association=evolution_oracle_diagonal_target)
    dbi(t_step, d = evolution_oracle_diagonal_target.h.dense.matrix)
    print(norm( dbi.h.exp(t_step) - u_frame_shifted))

In [ ]:
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation

In [ ]:
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.numerical
gc_numpy = gci.group_commutator( np.sqrt(t_step),
                                             diagonal_association_evolution_oracle= EvolutionOracle(d_0, "D0",mode_evolution_oracle=EvolutionOracleType.numerical))

In [ ]:
## Test more fancy functionalities
input_hamiltonian_evolution_oracle_hamiltonian_simulation.please_be_verbose = False
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(input_hamiltonian_evolution_oracle_hamiltonian_simulation ))
d_ev =  EvolutionOracle(d_0, "D0",mode_evolution_oracle=EvolutionOracleType.hamiltonian_simulation)

gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation
query_list = gci.group_commutator( np.sqrt(t_step),
                                             diagonal_association_evolution_oracle= d_ev )

norm(query_list['forwards'].unitary() -query_list['backwards'].unitary().conj().T)




In [ ]:
norm(query_list['forwards'] -query_list['backwards'].T.conj())

In [ ]:
#Test file entry
u = gci.iterated_hamiltonian_evolution_oracle.circuit( np.sqrt(t_step)).unitary()
u2 = gci.iterated_hamiltonian_evolution_oracle.circuit( -np.sqrt(t_step)).unitary()
norm(u-u2.T.conj())

In [ ]:
d_0.mode_evolution_oracle = EvolutionOracleType.text_strings
gci.iterated_hamiltonian_evolution_oracle.mode_evolution_oracle = EvolutionOracleType.text_strings
query_list = gci.group_commutator( np.sqrt(t_step*2),
             diagonal_association_evolution_oracle= EvolutionOracle(d_0, "D0"))


query_list['forwards']